-----

### How the Price-to-Sales ratio works

The price-to-sales ratio (Price/Sales or P/S) is calculated by taking a company's market capitalization (the number of outstanding shares multiplied by the share price) and divide it by the company's total sales or revenue over the past 12 months. The lower the P/S ratio, the more attractive the investment. Price-to-sales provides a useful measure for sizing up stocks.


> The price-to-sales ratio utilizes a company's market capitalization and revenue to determine whether the stock is valued properly.


                                                                                                         *investopedia.com*

#### Libraries

In [80]:
# Operating system
import os

# To create final df
import pandas as pd
from glob import glob
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Financial data
import yfinance as yf

#### Downloaded tickers


In [81]:
files = os.listdir('data/IncomeStatement/')

tickers_downloaded = set()

for f in files:
    tickers_downloaded.add(f.split(' ')[0].split('-')[0])
    
tickers = sorted(list(tickers_downloaded))

#### Reading file generated by notebook `01_Update_tickers`

In [83]:
# Leyendo archivo previamente generado
ticker_yf_data = pd.read_csv('data/all_tickers_yfinance_March2021.csv', index_col = 0)
ticker_yf_data.head()

,zip,sector,longBusinessSummary,city,phone,state,country,companyOfficers,website,maxAge,...,dateShortInterest,pegRatio,lastCapGain,shortPercentOfFloat,sharesShortPriorMonth,impliedSharesOutstanding,category,fiveYearAverageReturn,regularMarketPrice,logo_url
DDD,29730,Technology,"3D Systems Corporation, through its subsidiari...",Rock Hill,803 326 3930,SC,United States,[],http://www.3dsystems.com,1,...,1.613088e+09,15.36,NaN,0.1218,18478842.0,NaN,NaN,NaN,26.40,https://logo.clearbit.com/3dsystems.com
MMM,55144-1000,Industrials,"3M Company develops, manufactures, and markets...",St. Paul,651-733-1110,MN,United States,[],http://www.3m.com,1,...,1.613088e+09,2.52,NaN,0.0158,8139533.0,NaN,NaN,NaN,178.45,https://logo.clearbit.com/3m.com
WBAI,518115,Consumer Cyclical,"500.com Limited, through its subsidiaries, pro...",Shenzhen,86 755 8835 2500,NaN,China,[],http://www.500.com,1,...,1.613088e+09,NaN,NaN,NaN,2445891.0,NaN,NaN,NaN,19.48,https://logo.clearbit.com/500.com
EGHT,95008,Technology,"8x8, Inc. provides voice, video, chat, contact...",Campbell,408-727-1885,CA,United States,[],http://www.8x8.com,1,...,1.613088e+09,3.88,NaN,0.2138,20208760.0,NaN,NaN,NaN,33.42,https://logo.clearbit.com/8x8.com
AOS,53224-9508,Industrials,A. O. Smith Corporation manufactures and marke...,Milwaukee,414 359 4000,WI,United States,[],http://www.aosmith.com,1,...,1.613088e+09,2.86,NaN,0.0341,3903000.0,161468000.0,NaN,NaN,61.73,https://logo.clearbit.com/aosmith.com


In [84]:
# Se copia data para mantener archivo original porque se tarda en descargar...
ticker_data = ticker_yf_data.copy()

##### Selecting relevant columns

In [85]:
relevant_cols = ['shortName', 'marketCap', 'previousClose', 'fiftyDayAverage', 'averageVolume', 'sharesOutstanding',
                 'sector', 'industry']

ticker_data = ticker_data[relevant_cols]

In [86]:
ticker_data = ticker_data.loc[ list(set(ticker_data.index).intersection(set(tickers))) ]

Adding one hierarchical level *(makes merging easier)*

In [87]:
ticker_data.columns = pd.Index([('shortName', ''), ('marketCap', ''), ('previousClose', ''), ('fiftyDayAverage', ''),
                                   ('averageVolume', ''), ('sharesOutstanding', ''), ('sector', ''), ('industry', '')])

ticker_data

,shortName,marketCap,previousClose,fiftyDayAverage,averageVolume,sharesOutstanding,sector,industry
,,,,,,,,
FE,FirstEnergy Corp.,18040168448,33.22,32.222057,5055115,5.432150e+08,Utilities,Utilities—Diversified
BSX,Boston Scientific Corporation,55997341696,37.93,37.794117,11113676,NaN,NaN,NaN
IT,"Gartner, Inc.",16277734400,177.77,170.410300,445065,8.872160e+07,Technology,Information Technology Services
MRK,"Merck & Company, Inc.",185042288640,72.17,76.519090,10374976,2.530320e+09,Healthcare,Drug Manufacturers—General
DPZ,Domino's Pizza Inc,12814467072,330.53,369.492950,593215,3.880350e+07,Consumer Cyclical,Restaurants
...,...,...,...,...,...,...,...,...
LIN,Linde plc,129475108864,245.42,252.201170,1765915,5.253600e+08,Basic Materials,Specialty Chemicals
MCO,Moody's Corporation,53780025344,278.01,275.110600,779853,1.871000e+08,Financial Services,Financial Data & Stock Exchanges
GIS,"General Mills, Inc.",34717392896,55.29,56.781765,4254235,6.114370e+08,Consumer Defensive,Packaged Foods


### Including fundamental data (revenue)

In [88]:
def create_df_ticker(ticker_name, Statement_type, filename):
    """Given the name of the .csv file it returns a dataframe for a single ticker."""    
    # Read file
    income_ticker = pd.read_csv('data/' + Statement_type + '/' + filename, skiprows = 1)
    
    # Rename column
    income_ticker.columns = ['Data'] + list(income_ticker.columns[1:])
    # income_ticker.rename(columns = {'Fiscal year ends in September. USD in millions except per share data.': 'Data'}, inplace = True)
    
    # Fixing duplicated data names
    eps_index = income_ticker[income_ticker.iloc[:,0] == 'Earnings per share'].index[0]
    income_ticker.iloc[eps_index + 1, 0] += ' ' + income_ticker.iloc[eps_index, 0]
    income_ticker.iloc[eps_index + 2, 0] += ' ' + income_ticker.iloc[eps_index, 0]
    #income_ticker = pd.concat([income_ticker.iloc[:eps_index], income_ticker.iloc[eps_index+1:]])

    waso_index = income_ticker[income_ticker.iloc[:,0] == 'Weighted average shares outstanding'].index[0]
    income_ticker.iloc[waso_index + 1, 0] += ' ' + income_ticker.iloc[waso_index, 0]
    income_ticker.iloc[waso_index + 2, 0] += ' ' + income_ticker.iloc[waso_index, 0]
    #income_ticker = pd.concat([income_ticker.iloc[:waso_index], income_ticker.iloc[waso_index+1:]])
    income_ticker = income_ticker.set_index('Data').drop(['Operating expenses', 'Earnings per share', 'Weighted average shares outstanding'])

    
     # Change column names to eliminate month (we only care about years)
    income_ticker.columns = [x[:4] for x in income_ticker.columns]

    initial_year = int(income_ticker.columns[0])
    years_to_process = [str(initial_year + x) for x in range(1, 5)]  #skip first one...
    
    for x in ['Revenue']:
        
        if x not in income_ticker.index:
            income_ticker.loc[x] = np.nan  # Add full of NaN
        
        income_ticker.loc['Change % YoY {}'.format(x)] = np.nan
        for y in years_to_process:
            income_ticker.loc['Change % YoY ' + x, y] = (income_ticker.loc[x, y] / income_ticker.loc[x, str(int(y)-1)] - 1) * 100
    
        
    income_ticker.reset_index(inplace = True)
    
    # Transposing
    income_ticker = income_ticker.transpose()
    income_ticker.reset_index(inplace = True)

    # Renaming columns as the first row
    income_ticker.columns = income_ticker.iloc[0].values

    # Dropping first row
    income_ticker = income_ticker.iloc[1:]
    
    # Creating 'Year' and 'Ticker' columns
    income_ticker['Year'] = income_ticker['Data'].str[:4]
    income_ticker['Ticker'] = ticker_name
    
    # Setting 'Ticker' as index, dropping and sorting columns
    #income_ticker.set_index('Ticker', inplace = True)
    income_ticker.drop(['Data'], axis = 1, inplace = True)
    income_ticker = income_ticker.iloc[:, [income_ticker.shape[1] - 1, income_ticker.shape[1] - 2] + list(range(income_ticker.shape[1] - 2))  ]
    
    return income_ticker

In [89]:
def find_csv_name (ticker, statement_type, period = 'annual'):  # Falta ver que se necesita para quaterly
    """Find csv file name for a given ticker name, statement_type and period"""
    filenames = os.listdir('data/IncomeStatement/')
    for f in filenames:
        if ticker in f and statement_type in f and period in f:
            return f

## Main loop, putting together market and fundamental data.


In [90]:
for dat in ['Revenue', 'Cost of revenue']:
    i = True
    for t in tickers: 
        # Lectura de Internet si no está descargado ó si ya hay nuevo informe 
        
        # Crea df de ticker para dat
        inc_tick = create_df_ticker(t, 'IncomeStatement', find_csv_name(t, 'Income-Statement'))
        
        # En caso de que el dato que se quiera analizar no esté en el estado financiero
        if dat not in inc_tick:
            continue
        
        # Year como index
        inc_tick = inc_tick.set_index(['Year'])
        inc_tick.drop(['TTM'], inplace = True)
        
        years = list(inc_tick.index)
        # Añade un nivel jerárquico
        inc_tick.index = pd.MultiIndex.from_product([[dat], list(years)])
        
        # Transpone
        inc_tick = inc_tick.T
        
        # Localiza dato
        inc_tick_rev = inc_tick.loc[dat].to_frame().T
        
        # Renombra por ticker
        inc_tick_rev.rename(index = {dat: t}, inplace = True)
        
        # Une (o incluye) datos a tabla general
        if i: # Primer dato..
            ticker_data = ticker_data.merge(inc_tick_rev, how = 'left', left_index = True, right_index = True)
            i = False
        else:
            for y in years:
                ticker_data.loc[t, (dat, y)] = inc_tick_rev.loc[t, (dat, y)]
            
ticker_data

shortName     marketCap previousClose  \
                                                                  
FE                FirstEnergy Corp.  1.804017e+10         33.22   
BSX   Boston Scientific Corporation  5.599734e+10         37.93   
IT                    Gartner, Inc.  1.627773e+10        177.77   
MRK           Merck & Company, Inc.  1.850423e+11         72.17   
DPZ              Domino's Pizza Inc  1.281447e+10        330.53   
...                             ...           ...           ...   
TMUS              T-Mobile US, Inc.  1.549523e+11        118.61   
KR             Kroger Company (The)  2.622079e+10         34.09   
CTL                             NaN           NaN           NaN   
MYL                             NaN           NaN           NaN   
NBL                             NaN           NaN           NaN   

     fiftyDayAverage averageVolume sharesOutstanding                  sector  \
                                                                               
FE         32.222057     5055115.0      5.432150e+08               Utilities   
BSX        37.794117    11113676.0               NaN                     NaN   
IT        170.410300      445065.0      8.872160e+07              Technology   
MRK        76.519090    10374976.0      2.530320e+09              Healthcare   
DPZ       369.492950      593215.0      3.880350e+07       Consumer Cyclical   
...              ...           ...               ...                     ...   
TMUS      125.023636     4261733.0      1.242800e+09  Communication Services   
KR         33.777350    11550686.0      7.613470e+08      Consumer Defensive   
CTL              NaN           NaN               NaN                     NaN   
MYL              NaN           NaN               NaN                     NaN   
NBL              NaN           NaN               NaN                     NaN   

                             industry  Revenue                             \
                                          2016     2017     2018     2019   
FE              Utilities—Diversified  14562.0  14017.0  11261.0  11035.0   
BSX                               NaN   8386.0   9048.0   9823.0  10735.0   
IT    Information Technology Services   5783.0   5729.0   6151.0   7253.0   
MRK        Drug Manufacturers—General  39807.0  40122.0  42294.0  46840.0   
DPZ                       Restaurants   2473.0   2788.0   3433.0   3619.0   
...                               ...      ...      ...      ...      ...   
TMUS                 Telecom Services  37242.0  40604.0  43310.0  44998.0   
KR                     Grocery Stores  23110.0  17259.0  22877.0  23838.0   
CTL                               NaN     85.0     70.0     66.0     55.0   
MYL                               NaN      NaN      NaN      NaN      NaN   
NBL                               NaN  16608.0  16423.0  15360.0  15923.0   

                       Cost of revenue                                      \
         2020     2015            2016     2017     2018     2019     2020   
FE    10790.0      NaN          5479.0   4577.0   3647.0   3424.0   3070.0   
BSX    9913.0      NaN          2424.0   2593.0   2813.0   3116.0   3465.0   
IT        NaN   5982.0             NaN      NaN      NaN      NaN      NaN   
MRK       NaN  39498.0         13891.0  12775.0  13509.0  14112.0      NaN   
DPZ       NaN   2217.0          1705.0   1922.0   2130.0   2216.0      NaN   
...       ...      ...             ...      ...      ...      ...      ...   
TMUS  68397.0      NaN         16550.0  17708.0  18354.0  18521.0  28266.0   
KR        NaN  16688.0         20524.0  14046.0  18891.0  19406.0      NaN   
CTL      61.0      NaN            62.0     50.0     48.0     39.0     44.0   
MYL       NaN      NaN             NaN      NaN      NaN      NaN      NaN   
NBL       NaN  13414.0             NaN      NaN      NaN      NaN      NaN   

               
         2015  
FE        NaN  
BSX       NaN  
IT        NaN  
MRK   14934.0  


## Price to sales

In [91]:
ticker_data['Price/Sales (current)'] = ticker_data['sharesOutstanding'] * ticker_data['previousClose'] / ticker_data['Revenue', '2020']

In [92]:
ticker_data

shortName     marketCap previousClose  \
                                                                  
FE                FirstEnergy Corp.  1.804017e+10         33.22   
BSX   Boston Scientific Corporation  5.599734e+10         37.93   
IT                    Gartner, Inc.  1.627773e+10        177.77   
MRK           Merck & Company, Inc.  1.850423e+11         72.17   
DPZ              Domino's Pizza Inc  1.281447e+10        330.53   
...                             ...           ...           ...   
TMUS              T-Mobile US, Inc.  1.549523e+11        118.61   
KR             Kroger Company (The)  2.622079e+10         34.09   
CTL                             NaN           NaN           NaN   
MYL                             NaN           NaN           NaN   
NBL                             NaN           NaN           NaN   

     fiftyDayAverage averageVolume sharesOutstanding                  sector  \
                                                                               
FE         32.222057     5055115.0      5.432150e+08               Utilities   
BSX        37.794117    11113676.0               NaN                     NaN   
IT        170.410300      445065.0      8.872160e+07              Technology   
MRK        76.519090    10374976.0      2.530320e+09              Healthcare   
DPZ       369.492950      593215.0      3.880350e+07       Consumer Cyclical   
...              ...           ...               ...                     ...   
TMUS      125.023636     4261733.0      1.242800e+09  Communication Services   
KR         33.777350    11550686.0      7.613470e+08      Consumer Defensive   
CTL              NaN           NaN               NaN                     NaN   
MYL              NaN           NaN               NaN                     NaN   
NBL              NaN           NaN               NaN                     NaN   

                             industry  Revenue           ...           \
                                          2016     2017  ...     2019   
FE              Utilities—Diversified  14562.0  14017.0  ...  11035.0   
BSX                               NaN   8386.0   9048.0  ...  10735.0   
IT    Information Technology Services   5783.0   5729.0  ...   7253.0   
MRK        Drug Manufacturers—General  39807.0  40122.0  ...  46840.0   
DPZ                       Restaurants   2473.0   2788.0  ...   3619.0   
...                               ...      ...      ...  ...      ...   
TMUS                 Telecom Services  37242.0  40604.0  ...  44998.0   
KR                     Grocery Stores  23110.0  17259.0  ...  23838.0   
CTL                               NaN     85.0     70.0  ...     55.0   
MYL                               NaN      NaN      NaN  ...      NaN   
NBL                               NaN  16608.0  16423.0  ...  15923.0   

                       Cost of revenue                                      \
         2020     2015            2016     2017     2018     2019     2020   
FE    10790.0      NaN          5479.0   4577.0   3647.0   3424.0   3070.0   
BSX    9913.0      NaN          2424.0   2593.0   2813.0   3116.0   3465.0   
IT        NaN   5982.0             NaN      NaN      NaN      NaN      NaN   
MRK       NaN  39498.0         13891.0  12775.0  13509.0  14112.0      NaN   
DPZ       NaN   2217.0          1705.0   1922.0   2130.0   2216.0      NaN   
...       ...      ...             ...      ...      ...      ...      ...   
TMUS  68397.0      NaN         16550.0  17708.0  18354.0  18521.0  28266.0   
KR        NaN  16688.0         20524.0  14046.0  18891.0  19406.0      NaN   
CTL      61.0      NaN            62.0     50.0     48.0     39.0     44.0   
MYL       NaN      NaN             NaN      NaN      NaN      NaN      NaN   
NBL       NaN  13414.0             NaN      NaN      NaN      NaN      NaN   

              Price/Sales (current)  
         2015                        
FE        NaN        1672437.581346  
BSX       NaN                   Na